In [ ]:
%pip install kafka-python
%pip install confluent_kafka
%pip install pandas
%pip install pyarrow
%pip install fastparquet

In [7]:
import pandas as pd
from confluent_kafka import Producer
from confluent_kafka import Producer
from confluent_kafka.serialization import SerializationContext, MessageField
from confluent_kafka.schema_registry import SchemaRegistryClient
from confluent_kafka.schema_registry.json_schema import JSONSerializer
from confluent_kafka.serialization import StringSerializer, SerializationContext, MessageField
import socket
import time
from time import sleep
from json import dumps
import json
from datetime import timedelta
from io import BytesIO
from uuid import uuid4

In [8]:
df = pd.read_parquet('yellow_tripdata_2023-01.parquet')
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.30,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.90,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.90,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.10,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.40,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066761,2,2023-01-31 23:58:34,2023-02-01 00:12:33,NaN,3.05,NaN,None,107,48,0,15.80,0.00,0.5,3.96,0.0,1.0,23.76,NaN,NaN
3066762,2,2023-01-31 23:31:09,2023-01-31 23:50:36,NaN,5.80,NaN,None,112,75,0,22.43,0.00,0.5,2.64,0.0,1.0,29.07,NaN,NaN
3066763,2,2023-01-31 23:01:05,2023-01-31 23:25:36,NaN,4.67,NaN,None,114,239,0,17.61,0.00,0.5,5.32,0.0,1.0,26.93,NaN,NaN
3066764,2,2023-01-31 23:40:00,2023-01-31 23:53:00,NaN,3.15,NaN,None,230,79,0,18.15,0.00,0.5,4.43,0.0,1.0,26.58,NaN,NaN


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [10]:
df['tpep_pickup_datetime'] = df['tpep_pickup_datetime'].dt.strftime("%Y-%m-%d %H:%M:%S")
df['tpep_dropoff_datetime'] = df['tpep_dropoff_datetime'].dt.strftime("%Y-%m-%d %H:%M:%S") 

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   VendorID               int64  
 1   tpep_pickup_datetime   object 
 2   tpep_dropoff_datetime  object 
 3   passenger_count        float64
 4   trip_distance          float64
 5   RatecodeID             float64
 6   store_and_fwd_flag     object 
 7   PULocationID           int64  
 8   DOLocationID           int64  
 9   payment_type           int64  
 10  fare_amount            float64
 11  extra                  float64
 12  mta_tax                float64
 13  tip_amount             float64
 14  tolls_amount           float64
 15  improvement_surcharge  float64
 16  total_amount           float64
 17  congestion_surcharge   float64
 18  airport_fee            float64
dtypes: float64(12), int64(4), object(3)
memory usage: 444.6+ MB


In [32]:
dict_value = df.sample(1).to_dict(orient="records")[0]
type(dict_value['VendorID'])

int

In [12]:
def callback(err, event):
    if err:
        print(f'Produce to topic {event.topic()} failed for event: {event.key()}')
    else:
        val = event.value().decode('utf8')
        print(f'{val} sent to partition {event.partition()}.')

In [15]:
class Taxi_data(object):
    def __init__(self, VendorID, tpep_pickup_datetime, tpep_dropoff_datetime, passenger_count, 
                 trip_distance, RatecodeID, store_and_fwd_flag, PULocationID, DOLocationID, 
                 payment_type, fare_amount, extra, mta_tax, tip_amount,
                 tolls_amount, improvement_surcharge, total_amount, congestion_surcharge, airport_fee):
        self.VendorID = VendorID
        self.tpep_pickup_datetime = tpep_pickup_datetime
        self.tpep_dropoff_datetime = tpep_dropoff_datetime
        self.passenger_count = passenger_count
        self.trip_distance = trip_distance
        self.RatecodeID = RatecodeID
        self.store_and_fwd_flag = store_and_fwd_flag
        self.PULocationID = PULocationID
        self.DOLocationID = DOLocationID
        self.payment_type = payment_type
        self.fare_amount = fare_amount
        self.extra = extra
        self.mta_tax = mta_tax
        self.tip_amount = tip_amount
        self.tolls_amount = tolls_amount
        self.improvement_surcharge = improvement_surcharge
        self.total_amount = total_amount
        self.congestion_surcharge = congestion_surcharge
        self.airport_fee = airport_fee
        

In [16]:
def data_to_dict(taxi, ctx):
    """
    Returns a dict representation of a Taxi data instance for serialization.

    Args:
        taxi (Taxi): Taxi instance.

        ctx (SerializationContext): Metadata pertaining to the serialization
            operation.

    Returns:
        dict: Dict populated with user attributes to be serialized.
    """

    # User._address must not be serialized; omit from dict
    return dict(VendorID = taxi.VendorID,
        tpep_pickup_datetime = taxi.tpep_pickup_datetime,
        tpep_dropoff_datetime = taxi.tpep_dropoff_datetime,
        passenger_count = taxi.passenger_count,
        trip_distance = taxi.trip_distance,
        RatecodeID = taxi.RatecodeID,
        store_and_fwd_flag = taxi.store_and_fwd_flag,
        PULocationID = taxi.PULocationID,
        DOLocationID = taxi.DOLocationID,
        payment_type = taxi.payment_type,
        fare_amount = taxi.fare_amount,
        extra = taxi.extra,
        mta_tax = taxi.mta_tax,
        tip_amount = taxi.tip_amount,
        tolls_amount = taxi.tolls_amount,
        improvement_surcharge = taxi.improvement_surcharge,
        total_amount = taxi.total_amount,
        congestion_surcharge = taxi.congestion_surcharge,
        airport_fee = taxi.airport_fee)

In [38]:
schema_str = """{
    "$schema": "https://json-schema.org/draft/2020-12/schema",
    "title": "nyc_taxi",
    "description": "NYC Taxi Data",
    "type": "object",
    "properties": {
      "VendorID": {
        "description": "A code indicating the TPEP provider that provided the record.",
        "type": "number"
      },
      "tpep_pickup_datetime": {
        "description": "The date and time when the meter was engaged. ",
        "type": "string"
      },
      "tpep_dropoff_datetime": {
        "description": "The date and time when the meter was disengaged.",
        "type": "string"
      },
      "passenger_count": {
        "description": "The number of passengers in the vehicle. ",
        "type": "number"
      },
      "trip_distance": {
        "description": "The elapsed trip distance in miles reported by the taximeter.",
        "type": "number"
      },
      "RatecodeID": {
        "description": "The final rate code in effect at the end of the trip.",
        "type": "number"
      },
      "store_and_fwd_flag": {
        "description": "This flag indicates whether the trip record was held in vehicle memory before sending to the vendor, aka store and forward, because the vehicle did not have a connection to the server.",
        "type": "string"
      },
      "PULocationID": {
        "description": "TLC Taxi Zone in which the taximeter was engaged",
        "type": "number"
      },
      "DOLocationID": {
        "description": "TLC Taxi Zone in which the taximeter was disengaged",
        "type": "number"
      },
      "payment_type": {
        "description": "A numeric code signifying how the passenger paid for the trip.",
        "type": "number"
      },
      "fare_amount": {
        "description": "The time-and-distance fare calculated by the meter.",
        "type": "number"
      },
      "extra": {
        "description": "Miscellaneous extras and surcharges. Currently, this only includes the $0.50 and $1 rush hour and overnight charges.",
        "type": "number"
      },
      "mta_tax": {
        "description": "$0.50 MTA tax that is automatically triggered based on the metered rate in use.",
        "type": "number"
      },
      "tip_amount": {
        "description": "This field is automatically populated for credit card tips. Cash tips are not included.",
        "type": "number"
      },
      "tolls_amount": {
        "description": "Total amount of all tolls paid in trip.",
        "type": "number"
      },
      "improvement_surcharge": {
        "description": "$0.30 improvement surcharge assessed trips at the flag drop. The improvement surcharge began being levied in 2015.",
        "type": "number"
      },
      "total_amount": {
        "description": "The total amount charged to passengers. Does not include cash tips.",
        "type": "number"
      },
      "congestion_surcharge": {
        "description": "Total amount collected in trip for NYS congestion surcharge.",
        "type": "number"
      },
      "airport_fee": {
        "description": "$1.25 for pick up only at LaGuardia and John F. Kennedy Airports",
        "type": "number"
      }
    },
    "required": ["VendorID", "tpep_pickup_datetime", "tpep_dropoff_datetime"]
  }"""

In [44]:
def delivery_report(err, event):
    if err is not None:
        print(f'Delivery failed on reading for {event.key().decode("utf8")}: {err}')
    else:
        print(f'Temp reading for {event.key().decode("utf8")} produced to {event.topic()}')

def if_none(record):
    if record is None:
        record = ""
    return record

In [47]:
config = {'bootstrap.servers': 'localhost:9092',
        'group.id': 'test',
        'client.id': socket.gethostname()}
sr_config = {
    'url': 'http://localhost:8081'
}


In [53]:
if __name__ == '__main__':
    topic = 'hello_topic'
    schema_registry_client = SchemaRegistryClient(sr_config)

    string_serializer = StringSerializer('utf_8')
    json_serializer = JSONSerializer(schema_str,
                                     schema_registry_client,
                                     data_to_dict)

    producer = Producer(config)

    while True:
        try:
            dict_value = df.sample(1).to_dict(orient="records")[0]

            taxi_VendorID = if_none(dict_value['VendorID'])
            taxi_tpep_pickup_datetime = if_none(dict_value['tpep_pickup_datetime'])
            taxi_tpep_dropoff_datetime = if_none(dict_value['tpep_dropoff_datetime'])
            taxi_passenger_count = if_none(dict_value['passenger_count'])
            taxi_trip_distance = if_none(dict_value['trip_distance'])
            taxi_RatecodeID = if_none(dict_value['RatecodeID'])
            taxi_store_and_fwd_flag = if_none(dict_value['store_and_fwd_flag'])
            taxi_PULocationID = if_none(dict_value['PULocationID'])
            taxi_DOLocationID = if_none(dict_value['DOLocationID'])
            taxi_payment_type = if_none(dict_value['payment_type'])
            taxi_fare_amount = if_none(dict_value['fare_amount'])
            taxi_extra = if_none(dict_value['extra'])
            taxi_mta_tax = if_none(dict_value['mta_tax'])
            taxi_tip_amount = if_none(dict_value['tip_amount'])
            taxi_tolls_amount = if_none(dict_value['tolls_amount'])
            taxi_improvement_surcharge = if_none(dict_value['improvement_surcharge'])
            taxi_total_amount = if_none(dict_value['total_amount'])
            taxi_congestion_surcharge = if_none(dict_value['congestion_surcharge'])
            taxi_airport_fee = if_none(dict_value['airport_fee'])
            
            taxi_data = Taxi_data(VendorID = taxi_VendorID,
                tpep_pickup_datetime = taxi_tpep_pickup_datetime,
                tpep_dropoff_datetime = taxi_tpep_dropoff_datetime,
                passenger_count = taxi_passenger_count,
                trip_distance = taxi_trip_distance,
                RatecodeID = taxi_RatecodeID,
                store_and_fwd_flag = taxi_store_and_fwd_flag,
                PULocationID = taxi_PULocationID,
                DOLocationID = taxi_DOLocationID,
                payment_type = taxi_payment_type,
                fare_amount = taxi_fare_amount,
                extra = taxi_extra,
                mta_tax = taxi_mta_tax,
                tip_amount = taxi_tip_amount,
                tolls_amount = taxi_tolls_amount,
                improvement_surcharge = taxi_improvement_surcharge,
                total_amount = taxi_total_amount,
                congestion_surcharge = taxi_congestion_surcharge,
                airport_fee = taxi_airport_fee)
            
            producer.produce(topic=topic,
                            key=string_serializer(str(uuid4())),
                            value=json_serializer(taxi_data, SerializationContext(topic, MessageField.VALUE)),
                            on_delivery=delivery_report)
        except KeyboardInterrupt:
            break
        except ValueError:
            print("Invalid input, discarding record...")
            continue

    print("\nFlushing records...")
    producer.flush()

%4|1700991009.727|CONFWARN|LAPTOP-VCIHT22G#producer-19| [thrd:app]: Configuration property group.id is a consumer property and will be ignored by this producer instance



Flushing records...
Temp reading for eaa8d992-df10-420f-92db-8ad6cf419528 produced to hello_topic
Temp reading for 18157c69-f06c-41c7-91c2-6482d949eeb8 produced to hello_topic
Temp reading for 1f008c78-f991-4701-aac8-cf9eeb84f90f produced to hello_topic
Temp reading for 9e1d2daa-fa4c-456b-82f3-2738019c35cd produced to hello_topic
Temp reading for 67251372-20d5-4366-a5ea-40dcf95829a8 produced to hello_topic
Temp reading for 5005d808-eb64-42ef-b7a7-4526b485db8d produced to hello_topic
Temp reading for 085f5a71-6ee8-49bf-b764-a9fa83bf3328 produced to hello_topic
Temp reading for b1f24391-f2b7-43d7-93ae-6366247234a7 produced to hello_topic
Temp reading for f093b6a2-47e9-4d4e-9ba6-7a2448544891 produced to hello_topic
Temp reading for e21954f0-a172-4837-b9d5-bab49ad73dac produced to hello_topic
Temp reading for 8a0c2247-babd-43c6-9154-256ea9ff705d produced to hello_topic
Temp reading for f21939d3-dc27-4629-9d2d-8db79f4c8af0 produced to hello_topic
Temp reading for 9e89cfcf-7fc1-43ef-a434-1e